<h2><font color="#004D7F" size=5>Módulo 1: Modelos múltiples</font></h2>



<h1><font color="#004D7F" size=6> 4. Selección de clasificador dinámico </font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Aprendizaje Automático II</font><br>
<font color="#004D7F" size=3>Universidad Nacional de Educación a Distancia</font>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>


* [1. Selección del clasificador dinámico](#section1)
   * [1.1. Librería DESlib](#section11)
   * [1.2. Dataset](#section12)
* [2. Selección de Clasificadores dinámicos (DCS)](#section2)
    * [2.1. DCS-LA con accuracy local general (OLA)](#section21)
    * [2.2. DCS-LA con accuracy de clase local (LCA)](#section22)
* [3. Ajuste de hiperparámetros para DCS](#section3)
    * [3.1. Parámetro `k` en KNN](#section31)
    * [3.2. Explorar algoritmos para el grupo de clasificadores](#section32)
* [Ejercicios](#sectionEj)

---

<a id="section0"></a>
# <font color="#004D7F">0. Contexto</font>

La técnica implica ajustar múltiples modelos de aprendizaje automático en el conjunto de datos de entrenamiento y luego seleccionar el modelo que se espera que funcione mejor al hacer una predicción, en función de los detalles específicos del ejemplo que se va a predecir. Por tanto, en este tutorial trabajaremos:
- Los algoritmos de selección de clasificador dinámico eligen uno entre muchos modelos para hacer una predicción para cada nuevo ejemplo.
- Cómo desarrollar y evaluar modelos de selección de clasificadores dinámicos para tareas de clasificación utilizando Scikit-learn.
- Cómo explorar el efecto de los hiperparámetros del modelo de selección de clasificador dinámico clasificación.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section1"></a>
# <font color="#004D7F"> 1. Selección del clasificador dinámico</font>

__Selección de clasificador dinámico (DCS)__: 
- Son algoritmos que eligen uno de entre muchos modelos entrenados para hacer una predicción basada en los detalles específicos de la entrada.
- Los algoritmos DCS generalmente implican dividir el espacio de características de entrada de alguna manera y asignar modelos específicos que sean responsables de hacer predicciones para cada partición.
- Los esfuerzos de investigación se centran principalmente en cómo evaluar y asignar clasificadores a regiones específicas del espacio de entrada.

```text
Después de capacitar a varios clasificadores individuales, DCS selecciona dinámicamente un clasificador para cada instancia de prueba. [...] DCS hace predicciones utilizando un clasificador individual.
        -- Página 93, Ensemble Methods: Foundations and Algorithms, 2012.
```
Enfoque __Dynamic Classifier Selection Local Accuracy (DCS-LA)__ (del artículo _Combination Of Multiple Classifiers Using Local Accuracy Estimates_, 1997):
- Primero adapta un conjunto pequeño y diverso de modelos de clasificación al conjunto de datos de entrenamiento.
- Cuando se requiere una predicción, primero se utiliza un algoritmo KNN para encontrar los _k_ ejemplos más similares del conjunto de datos de entrenamiento que coincidan con el ejemplo.
- Luego, cada clasificador previamente ajustado en el modelo se evalúa en el vecino de _k_ ejemplos de entrenamiento y se selecciona el clasificador que funciona mejor para hacer una predicción para el nuevo ejemplo.
- "La idea básica es estimar el accuracy de cada clasificador en la región local del espacio de características que rodea una muestra de test desconocida y luego utilizar la decisión del clasificador más preciso localmente." 

Dos enfoques para seleccionar un modelo:
- __Accuracy local (LA u OLA)__,:
    1. Evalua el accuracy de la clasificación de cada modelo en la vecindad de _k_ ejemplos de entrenamiento.
    2. Luego se selecciona el modelo que funciona mejor en este vecindario para hacer una predicción para el nuevo ejemplo.
- __Accuracy de clase (CA o LCA)__:
    1. Utiliza cada modelo para hacer una predicción para el nuevo ejemplo y anotar la clase que se predijo.
    2. Luego, se evalúa el accuracy de cada modelo en el vecino de _k_ ejemplos de entrenamiento y
    3. Se selecciona el modelo que tiene la mejor habilidad para la clase que predijo en el nuevo ejemplo y se devuelve su predicción.

<a id="section11"></a>
## <font color="#004D7F"> 1.1. Librería DESlib</font>

_Dynamic ensemble selection library in Python_ (DESlib) es una biblioteca de código abierto que proporciona una implementación de muchos algoritmos diferentes de selección de clasificadores dinámicos. 

Instalamos `deslib`.

In [3]:
#!pip install deslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.9/158.9 kB 940.5 kB/s eta 0:00:00a 0:00:01


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre la librería [DESlib](https://deslib.readthedocs.io/en/latest/).
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre la librería [DESlib](https://deslib.readthedocs.io/en/latest/).
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre el Github [DESlib](https://github.com/scikit-learn-contrib/DESlib).
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información en PyPI sobre [DESlib](https://pypi.org/project/DESlib/0.1/).
</div>

In [4]:
# check deslib version
import deslib
print(deslib.__version__)

0.3.5


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Puede que tenga que reiniciar el kernel para que pueda ejecutarse correctamente la librería.
</div>

DESlib proporciona una implementación del algoritmo DCS-LA con cada técnica de selección de clasificador a través de las clases OLA y LCA. 
- Cada clase se puede utilizar directamente como modelo de Scikit-learn.
- Ambas clases utilizan un algoritmo KNN para seleccionar el vecino con un valor predeterminado de `k = 7`. 
- Se utiliza Bagging como conjunto de modelos considerados para cada clasificación que se realiza.

<a id="section12"></a>
## <font color="#004D7F"> 1.2. Dataset</font>

Usamos `make_classification()` para crear un problema de clasificación binaria sintética con 10,000 ejemplos y 20 características de entrada.

In [1]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
print(X.shape, y.shape)

(10000, 20) (10000,)


---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section2"></a> 
# <font color="#004D7F"> 2. Selección de Clasificadores dinámicos (DCS) </font>

Veamos cómo utilizar cada algoritmo DCS-LA y DCS-LCA.

<a id="section21"></a> 
## <font color="#004D7F"> 2.1. DCS-LA con accuracy local general (OLA)</font>

Podemos evaluar un modelo DCS-LA utilizando el accuracy local en el conjunto de datos sintéticos. 

<a id="section211"></a> 
### <font color="#004D7F"> 2.1.1. Evaluación</font>

Veamos la evaluación:
- Usaremos hiperparámetros por defecto: Bagging como conjunto de modelos clasificadores y `k = 7` para la vecindad local al hacer una predicción. 
- Utilizamos una validación cruzada estratificada repetida de _k_ veces con tres repeticiones y 10 pliegues. 

In [2]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from deslib.dcs.ola import OLA

# definir el modelo
model = ???
# Validación cruzada
cv = ???
# evaluación
n_scores = ???
print('Accuracy medio: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

SyntaxError: invalid syntax (1090382519.py, line 8)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section212"></a> 
### <font color="#004D7F"> 2.1.2. Predicción</font>

También podemos utilizar el modelo DCS-LA con OLA como modelo final y hacer predicciones para la clasificación. Primero, el modelo se ajusta a todos los datos disponibles, luego se puede llamar a la función `predict()` para hacer predicciones sobre nuevos datos. 

In [3]:
from deslib.dcs.ola import OLA

# denifinir el modelo
model = ???
# entrenamiento
???
# Hacer una predicción
row = [1.89149379, -0.39847585, 1.63856893, 0.01647165, 1.51892395, -3.52651223, 1.80998823, 0.58810926, -0.02542177, -0.52835426]
pred = ???
# summarize the prediction
print('Predicción: %d' % pred[0])

SyntaxError: invalid syntax (3803796519.py, line 4)

<a id="section22"></a> 
## <font color="#004D7F"> 2.2. DCS-LA con accuracy de clase local (LCA)</font>

Podemos evaluar un modelo DCS-LA con LCA utilizando la precisión de clase local en el conjunto de datos sintéticos.

<a id="section221"></a> 
### <font color="#004D7F"> 2.2.1. Evaluación</font>

Veamos la evaluación:
- Usaremos hiperparámetros por defecto: Bagging como conjunto de modelos clasificadores y `k = 7` para la vecindad local al hacer una predicción. 
- Utilizamos una validación cruzada estratificada repetida de _k_ veces con tres repeticiones y 10 pliegues. 

In [7]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from deslib.dcs.lca import LCA

# definir el modelo
model = ???

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 0.919 (0.009)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section212"></a> 
### <font color="#004D7F"> 2.1.2. Predicción</font>

También podemos utilizar el modelo DCS-LA con LCA como modelo final y hacer predicciones para la clasificación. Primero, el modelo se ajusta a todos los datos disponibles, luego se puede llamar a la función `predict()` para hacer predicciones sobre nuevos datos. 

El siguiente ejemplo demuestra esto en nuestro conjunto de datos de clasificación binaria.

In [4]:
# make a prediction with DCS-LA using local class accuracy
from sklearn.datasets import make_classification
from deslib.dcs.lca import LCA

# ddefinir el modelo
model = ???

model.fit(X, y)
row = [0.2929949, -4.21223056, -1.288332, -2.17849815, -0.64527665, 2.58097719, 0.28422388, -7.1827928, -1.91211104, 2.73729512, 0.81395695, 3.96973717, -2.66939799, 3.34692332, 4.19791821, 0.99990998, -0.30201875, -4.43170633, -2.82646737, 0.44916808]
pred = model.predict([row])
print('Predicción: %d' % pred[0])

SyntaxError: invalid syntax (1106155019.py, line 6)

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section3"></a> 
# <font color="#004D7F"> 3. Ajuste de hiperparámetros para DCS</font>

En esta sección, analizaremos dos de los principales hiperparámetros:
- El valor de `k` en el modelo KNN utilizado en la evaluación local de los modelos, y
- Cómo usar un hiperparámetro personalizado para el conjunto de clasificadores.

Usaremos DCS-LA con OLA como base para estos experimentos.

<a id="section31"></a> 
## <font color="#004D7F"> 3.1. Parámetro `k` en KNN</font>

La configuración del algoritmo de KNN es fundamental para el modelo DCS-LA ya que define el alcance de la vecindad en la que se considera cada clasificador para la selección. El valor `k` controla el tamaño de la vecindad:
- Valores demasiado pequeños significará que los ejemplos relevantes en el conjunto de entrenamiento podrían excluirse de la vecindad.
- Valores demasiado grandes pueden significar que la señal está siendo eliminada por demasiados ejemplos. 

Veamos un ejemplo de DCS-LA con OLA para valores `k` de 2 a 21.

In [5]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from deslib.dcs.ola import OLA
from matplotlib import pyplot

def get_dataset():
    X, y = make_classification(n_samples=10000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
    return X, y

# lista de modelos a evaluar (valor e k de 2 a 21)
def get_models():
    ???
    ???
        ???
    ???

# evaluar el modelo
def evaluate_model(model, X, y):
    ???
    ???
    ???

# dataset
X, y = ???
# obtener modelos a evaluar
models = ???
# evaluar y almacenar resultados
results, names = ???
???
    ???
    ???
    ???
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

IndentationError: unexpected indent (727423868.py, line 17)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section32"></a> 
## <font color="#004D7F"> 3.2. Explorar algoritmos para el grupo de clasificadores</font>

De forma predeterminada se utilizan árboles de decisión Bagging.

<a id="section321"></a> 
### <font color="#004D7F"> 3.2.1. Grupo de clasificadores</font>

Se puede considerar un grupo personalizado de clasificadores. 
- Como no podemos utilizar una validación cruzada vamos a utilizar un partición por porcentaje.
- La lista de clasificadores aptos se puede especificar para la clase OLA (o LCA) a través del argumento `pool_classifiers`.
- En este caso, utilizaremos un grupo que incluye regresión logística, un árbol de decisión y Naive Bayes. 

In [6]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from deslib.dcs.ola import OLA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

X, y = get_dataset()
# Particiòn en un 50%
???
# definimos los clasificadores
classifiers = ???
# Entrenamos cada clasificador en train
???
# definimos el modelo DCS-LA
???
# entrenamos el metamodelo
???
# hacer las predicciónes en test y sacar el accuracy
???)
# evaluate predictions
score = ???)
print('Accuracy: %.3f' % (score))

SyntaxError: unmatched ')' (3948550307.py, line 23)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section322"></a> 
### <font color="#004D7F"> 3.2.2. Modelo contribuyente</font>

- Para adoptar el modelo DCS, debe funcionar mejor que cualquier modelo contribuyente.
- De lo contrario, simplemente utilizaríamos el modelo contribuyente que funcione mejor.
- Podemos verificar esto evaluando el desempeño de cada clasificador contribuyente en test.

In [7]:
# evaluate contributing models
print('Accuracy total: %.3f' % (score))
???
    ???
    ???
    print('>%s: %.3f' % (c.__class__.__name__, score))

IndentationError: unexpected indent (4289001939.py, line 4)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="sectionEj"></a>
<h3><font color="#004D7F" size=6> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicios</font></h3>

Se proponen las siguientes actividades para consolidar el aprendizaje.

# <font color="#004D7F" size=5>Ejercicio 1</font>
__DCS-LA con LCA__. En los ejemplos finales hemos visto para OLA, utilice el ejemplo con LCA.

# <font color="#004D7F" size=5>Ejercicio 2</font>
__Hiperparámetros__. ¿Qué otros hiperparámetros se pueden utilizar? Indaga y descubra más hiperparámetros y evalúe su uso.

# <font color="#004D7F" size=5>Ejercicio 3</font>
__Más conjuntos de datos__. Busque un dataset original y verdadero (que no sea sintético) y evalúe el uso de los conceptos vistos en esta unidad. Los conjuntos de datos en pueden ser obtenidos del [repositorio de aprendizaje automático de UCI](https://archive.ics.uci.edu/).

# <font color="#004D7F" size=5>Ejercicio 4</font>
__Clasificadores__. Evalué todo los ejercicios anteriores con el uso de clasificadores distintos a los vistos en la unidad, i.e., diferentes a LoR, CART y NB.

# <font color="#004D7F" size=5>Ejercicio 5</font>
__Más sobre DESlib__. DESlib es una librería muy interesante que incluye una gran cantidad de modelos y técnicas. Selecciona 4 técnicas adicionales y desarróllelas (tanto teóricamente de la técnica como la práctica).

---

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>